# MRZ CRNN Training (v2: Enhanced)

MRZ (Machine Readable Zone) 用の軽量 CRNN モデルを学習する。

## v2 改善点
- **データ拡張強化**: ノイズ、ブラー、回転、Perspective変換、輝度変動
- **Attention機構追加**: Self-Attention で文字間依存関係を学習
- **正則化強化**: Dropout増加、Weight Decay調整

## 概要
- **アーキテクチャ**: CNN + BiLSTM + **Self-Attention** + CTC
- **入力**: 32x280 グレースケール画像
- **出力**: 44文字の MRZ テキスト
- **目標**: CER < 0.5%, Accuracy >= 99%

## 1. セットアップ

In [ ]:
# GPU 確認
!nvidia-smi

In [ ]:
# 必要なライブラリは Colab に事前インストール済み
import torch
import torch.nn as nn
import numpy as np
from pathlib import Path
import random
import time

# cuDNN最適化（低リスク高速化）
torch.backends.cudnn.benchmark = True  # 最適なアルゴリズム自動選択
torch.backends.cudnn.deterministic = False  # 再現性より速度優先

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"cuDNN benchmark: {torch.backends.cudnn.benchmark}")

## 2. OCR-B フォント準備 & 合成 MRZ データ生成

MRZ は **OCR-B フォント** で印刷されている。
合成データも OCR-B を使用して実際の MRZ に近い画像を生成する。

In [ ]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageEnhance
import string
import io

# MRZ で使用する文字セット
MRZ_CHARS = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789<"

def generate_random_mrz_line() -> str:
    """
    ランダムな MRZ 行（44文字）を生成
    
    実際の MRZ フォーマットに近いパターンを生成:
    - Line 1: P<COUNTRY_CODE + NAME + FILLER
    - Line 2: PASSPORT_NO + CHECK + NATIONALITY + DOB + CHECK + SEX + EXPIRY + CHECK + OPTIONAL + CHECK
    """
    if random.random() < 0.5:
        # Line 1 形式
        doc_type = random.choice(["P", "I", "A", "C"])
        country = "".join(random.choices(string.ascii_uppercase, k=3))
        name_len = random.randint(20, 35)
        name = "".join(random.choices(string.ascii_uppercase + "<", k=name_len))
        line = f"{doc_type}<{country}{name}"
        line = line[:44].ljust(44, "<")
    else:
        # Line 2 形式
        passport_no = "".join(random.choices(string.ascii_uppercase + string.digits, k=9))
        check1 = random.choice(string.digits)
        nationality = "".join(random.choices(string.ascii_uppercase, k=3))
        dob = "".join(random.choices(string.digits, k=6))
        check2 = random.choice(string.digits)
        sex = random.choice(["M", "F", "<"])
        expiry = "".join(random.choices(string.digits, k=6))
        check3 = random.choice(string.digits)
        optional = "".join(random.choices(string.ascii_uppercase + string.digits + "<", k=14))
        check4 = random.choice(string.digits)
        line = f"{passport_no}{check1}{nationality}{dob}{check2}{sex}{expiry}{check3}{optional}{check4}"
        line = line[:44]
    
    return line


def apply_augmentation(img: Image.Image) -> Image.Image:
    """
    データ拡張を適用（v2 強化版）
    
    実際のスキャン/カメラ撮影で発生する劣化をシミュレート:
    - ガウシアンブラー: フォーカスぼけ
    - 回転: 微小な傾き
    - Perspective変換: 斜め撮影
    - 輝度/コントラスト変動: 照明条件
    - ノイズ: センサーノイズ
    - JPEG圧縮: 圧縮アーティファクト
    """
    # 1. ガウシアンブラー (30%の確率)
    if random.random() < 0.3:
        radius = random.uniform(0.3, 1.0)
        img = img.filter(ImageFilter.GaussianBlur(radius=radius))
    
    # 2. 回転 (50%の確率、±2度)
    if random.random() < 0.5:
        angle = random.uniform(-2, 2)
        img = img.rotate(angle, fillcolor=255, resample=Image.BILINEAR)
    
    # 3. Perspective変換 (20%の確率)
    if random.random() < 0.2:
        w, h = img.size
        dx = random.uniform(-0.02, 0.02) * w
        dy = random.uniform(-0.02, 0.02) * h
        coeffs = [
            1 + random.uniform(-0.01, 0.01),
            random.uniform(-0.02, 0.02),
            dx,
            random.uniform(-0.02, 0.02),
            1 + random.uniform(-0.01, 0.01),
            dy,
            0, 0
        ]
        img = img.transform((w, h), Image.AFFINE, coeffs[:6], fillcolor=255)
    
    # 4. 輝度変動 (40%の確率)
    if random.random() < 0.4:
        factor = random.uniform(0.8, 1.2)
        enhancer = ImageEnhance.Brightness(img)
        img = enhancer.enhance(factor)
    
    # 5. コントラスト変動 (40%の確率)
    if random.random() < 0.4:
        factor = random.uniform(0.8, 1.2)
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(factor)
    
    # 6. JPEG圧縮シミュレーション (20%の確率)
    if random.random() < 0.2:
        quality = random.randint(70, 95)
        buffer = io.BytesIO()
        img.save(buffer, format='JPEG', quality=quality)
        buffer.seek(0)
        img = Image.open(buffer).convert('L')
    
    return img


def render_mrz_image(
    text: str,
    height: int = 32,
    font_size: int = 24,
    augment: bool = True
) -> np.ndarray:
    """
    MRZ テキストを OCR-B フォントで画像にレンダリング（v2 強化版）
    
    Args:
        text: 44文字の MRZ テキスト
        height: 出力画像の高さ
        font_size: フォントサイズ
        augment: データ拡張を適用するか
    
    Returns:
        グレースケール画像 (H, W)
    """
    # OCR-B フォントを使用（MRZ標準フォント）
    try:
        font = ImageFont.truetype(OCRB_FONT_PATH, font_size)
    except:
        # フォールバック: モノスペースフォント
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSansMono.ttf", font_size)
        except:
            font = ImageFont.load_default()
    
    # テキストサイズを計算
    dummy_img = Image.new("L", (1, 1))
    dummy_draw = ImageDraw.Draw(dummy_img)
    bbox = dummy_draw.textbbox((0, 0), text, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    
    # 画像を作成（白背景）
    padding = 4
    img_width = text_width + padding * 2
    img = Image.new("L", (img_width, height), color=255)
    draw = ImageDraw.Draw(img)
    
    # テキストを描画（黒文字）
    y_offset = (height - text_height) // 2
    draw.text((padding, y_offset), text, font=font, fill=0)
    
    # データ拡張を適用
    if augment:
        img = apply_augmentation(img)
    
    # NumPy 配列に変換
    img_array = np.array(img)
    
    # ガウシアンノイズ (60%の確率)
    if augment and random.random() < 0.6:
        sigma = random.uniform(3, 10)
        noise = np.random.normal(0, sigma, img_array.shape)
        img_array = np.clip(img_array + noise, 0, 255).astype(np.uint8)
    
    return img_array


# テスト
print("データ拡張テスト (v2 強化版 + OCR-B フォント)")
sample_text = generate_random_mrz_line()
sample_img = render_mrz_image(sample_text, augment=True)
print(f"Sample MRZ: {sample_text}")
print(f"Image shape: {sample_img.shape}")

In [ ]:
# サンプル画像を表示
import matplotlib.pyplot as plt

fig, axes = plt.subplots(3, 1, figsize=(12, 4))
for i, ax in enumerate(axes):
    text = generate_random_mrz_line()
    img = render_mrz_image(text)
    ax.imshow(img, cmap="gray")
    ax.set_title(text, fontsize=8)
    ax.axis("off")
plt.tight_layout()
plt.show()

## 3. Dataset & DataLoader

In [ ]:
from torch.utils.data import Dataset, DataLoader

# 文字セット定義
CHARS = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789<"
CHAR_TO_IDX = {c: i for i, c in enumerate(CHARS)}
IDX_TO_CHAR = {i: c for i, c in enumerate(CHARS)}
NUM_CLASSES = len(CHARS) + 1  # +1 for CTC blank

print(f"文字数: {len(CHARS)}")
print(f"クラス数 (blank含む): {NUM_CLASSES}")


def encode_text(text: str) -> list:
    """テキストを数値インデックスに変換"""
    return [CHAR_TO_IDX[c] for c in text if c in CHAR_TO_IDX]


def decode_output(indices: list) -> str:
    """
    CTC 出力をテキストにデコード
    連続する同一インデックスと blank を除去
    """
    result = []
    prev_idx = -1
    for idx in indices:
        if idx == len(CHARS):  # blank
            prev_idx = idx
            continue
        if idx != prev_idx and idx < len(CHARS):
            result.append(IDX_TO_CHAR[idx])
        prev_idx = idx
    return "".join(result)


class SyntheticMRZDataset(Dataset):
    """
    合成 MRZ データセット (v2)
    
    オンラインでランダムに MRZ 画像を生成する。
    epoch ごとに異なるデータが生成される。
    OCR-B フォント + データ拡張強化。
    """
    
    def __init__(self, num_samples: int, max_width: int = 280):
        self.num_samples = num_samples
        self.max_width = max_width
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        # ランダムな MRZ を生成
        text = generate_random_mrz_line()
        image = render_mrz_image(text, augment=True)  # v2: augment引数
        
        # 正規化 (0-1)
        image = image.astype(np.float32) / 255.0
        
        # パディング（幅を max_width に統一）
        h, w = image.shape
        if w < self.max_width:
            pad_w = self.max_width - w
            image = np.pad(image, ((0, 0), (0, pad_w)), constant_values=1.0)
        elif w > self.max_width:
            image = image[:, :self.max_width]
        
        # テンソルに変換 (1, H, W)
        image_tensor = torch.from_numpy(image).unsqueeze(0)
        label = encode_text(text)
        
        return {
            "image": image_tensor,
            "label": torch.tensor(label, dtype=torch.long),
            "label_length": len(label),
            "text": text
        }


def collate_fn(batch):
    """バッチをまとめる（CTC Loss 用）"""
    images = torch.stack([item["image"] for item in batch])
    labels = torch.cat([item["label"] for item in batch])
    label_lengths = torch.tensor([item["label_length"] for item in batch])
    texts = [item["text"] for item in batch]
    return {
        "images": images,
        "labels": labels,
        "label_lengths": label_lengths,
        "texts": texts
    }

## 4. CRNN モデル

In [ ]:
class AttentionCRNN(nn.Module):
    """
    Attention-enhanced CRNN for MRZ OCR (v2)
    
    アーキテクチャ:
    - CNN Backbone: 特徴抽出（高さを1に圧縮）
    - BiLSTM: シーケンスモデリング
    - Self-Attention: 文字間依存関係の学習（類似文字 0/O, 1/I の識別向上）
    - Linear: 文字分類（37クラス + CTC blank）
    
    入力: (B, 1, 32, W) - グレースケール画像
    出力: (T, B, 38) - 各タイムステップの文字確率
    
    v2 改善点:
    - Self-Attention追加（num_heads=4）
    - Dropout強化（0.1 → 0.2）
    - LayerNorm追加で学習安定化
    """
    
    def __init__(self, num_classes: int = 38, hidden_size: int = 128, dropout: float = 0.2):
        super().__init__()
        
        self.hidden_size = hidden_size
        
        # CNN Backbone（変更なし）
        self.cnn = nn.Sequential(
            # Block 1: 32 -> 16
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            # Block 2: 16 -> 8
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            # Block 3: 8 -> 4
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),
            
            # Block 4: 4 -> 2
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),
            
            # Block 5: 2 -> 1
            nn.Conv2d(256, 256, kernel_size=(2, 1)),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )
        
        # BiLSTM（Dropout強化）
        self.lstm = nn.LSTM(
            input_size=256,
            hidden_size=hidden_size,
            num_layers=2,
            bidirectional=True,
            batch_first=False,
            dropout=dropout
        )
        
        # Self-Attention (v2 追加)
        # BiLSTM出力に対してSelf-Attentionを適用
        # 文字間の依存関係を学習し、類似文字の識別精度を向上
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_size * 2,  # BiLSTM出力は hidden_size * 2
            num_heads=4,
            dropout=dropout,
            batch_first=False  # (T, B, C) 形式
        )
        
        # LayerNorm（Attention後の正規化で学習安定化）
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        
        # Dropout層
        self.dropout = nn.Dropout(dropout)
        
        # 出力層
        self.fc = nn.Linear(hidden_size * 2, num_classes)
    
    def forward(self, x):
        # CNN 特徴抽出: (B, 1, 32, W) -> (B, 256, 1, W')
        features = self.cnn(x)
        
        # 形状変換: (B, C, 1, W') -> (W', B, C) = (T, B, C)
        b, c, h, w = features.shape
        features = features.squeeze(2)
        features = features.permute(2, 0, 1)  # (T, B, C)
        
        # BiLSTM: (T, B, 256) -> (T, B, hidden*2)
        lstm_out, _ = self.lstm(features)
        
        # Self-Attention: (T, B, C) -> (T, B, C)
        # 文字間の依存関係を学習
        attn_out, _ = self.attention(lstm_out, lstm_out, lstm_out)
        
        # Residual connection + LayerNorm
        # 元のLSTM出力とAttention出力を足し合わせて安定化
        out = self.layer_norm(lstm_out + attn_out)
        out = self.dropout(out)
        
        # 出力層: (T, B, hidden*2) -> (T, B, num_classes)
        output = self.fc(out)
        output = torch.log_softmax(output, dim=2)
        
        return output


# 旧モデルも残しておく（比較用）
class CRNN(nn.Module):
    """旧バージョン（Attentionなし）"""
    def __init__(self, num_classes: int = 38, hidden_size: int = 128):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.Conv2d(256, 256, kernel_size=(2, 1)), nn.BatchNorm2d(256), nn.ReLU(),
        )
        self.lstm = nn.LSTM(256, hidden_size, 2, bidirectional=True, batch_first=False, dropout=0.1)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
    
    def forward(self, x):
        features = self.cnn(x)
        b, c, h, w = features.shape
        features = features.squeeze(2).permute(2, 0, 1)
        lstm_out, _ = self.lstm(features)
        output = self.fc(lstm_out)
        return torch.log_softmax(output, dim=2)


def get_model_info(model):
    """モデル情報を取得"""
    total_params = sum(p.numel() for p in model.parameters())
    param_size = sum(p.numel() * p.element_size() for p in model.parameters())
    buffer_size = sum(b.numel() * b.element_size() for b in model.buffers())
    size_mb = (param_size + buffer_size) / 1024 / 1024
    return {"total_params": total_params, "size_mb": size_mb}


# AttentionCRNN（v2）を使用
model = AttentionCRNN(num_classes=NUM_CLASSES, dropout=0.2)
info = get_model_info(model)
print(f"モデル: AttentionCRNN (v2)")
print(f"パラメータ数: {info['total_params']:,}")
print(f"モデルサイズ: {info['size_mb']:.2f} MB")

# 推論テスト
x = torch.randn(1, 1, 32, 280)
output = model(x)
print(f"入力: {x.shape} -> 出力: {output.shape}")

## 5. 学習

In [ ]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR

# ハイパーパラメータ (v2)
BATCH_SIZE = 64
EPOCHS = 100
LR = 1e-3
WEIGHT_DECAY = 0.01  # v2: 正則化強化
TRAIN_SAMPLES = 10000
VAL_SAMPLES = 1000
MAX_WIDTH = 280

# デバイス
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# データセット
train_dataset = SyntheticMRZDataset(TRAIN_SAMPLES, MAX_WIDTH)
val_dataset = SyntheticMRZDataset(VAL_SAMPLES, MAX_WIDTH)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=2
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=2
)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

# モデル・損失関数・オプティマイザ (v2: AttentionCRNN使用)
model = AttentionCRNN(num_classes=NUM_CLASSES, dropout=0.2).to(device)
criterion = nn.CTCLoss(blank=NUM_CLASSES - 1, zero_infinity=True)
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

# OneCycleLR: Warmup → 高学習率 → 低学習率
scheduler = OneCycleLR(
    optimizer,
    max_lr=LR * 10,           # 最大学習率 1e-2
    epochs=EPOCHS,
    steps_per_epoch=len(train_loader),
    pct_start=0.1,            # 最初の10%で warmup
    anneal_strategy='cos'
)

print(f"\n[v2 設定]")
print(f"モデル: AttentionCRNN (Self-Attention + Dropout 0.2)")
print(f"Weight Decay: {WEIGHT_DECAY}")
print(f"データ拡張: ノイズ、ブラー、回転、Perspective、輝度変動、JPEG圧縮")

In [ ]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR

# ハイパーパラメータ (v2 高速化版)
BATCH_SIZE = 128  # 64 → 128 (GPU活用率向上)
EPOCHS = 100
LR = 1e-3
WEIGHT_DECAY = 0.01
TRAIN_SAMPLES = 10000
VAL_SAMPLES = 1000
MAX_WIDTH = 280

# デバイス
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# データセット
train_dataset = SyntheticMRZDataset(TRAIN_SAMPLES, MAX_WIDTH)
val_dataset = SyntheticMRZDataset(VAL_SAMPLES, MAX_WIDTH)

# DataLoader (高速化設定)
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=4,      # 2 → 4
    pin_memory=True,    # GPU転送高速化
    persistent_workers=True  # ワーカー再利用
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

# モデル
model = AttentionCRNN(num_classes=NUM_CLASSES, dropout=0.2).to(device)

# torch.compile() で高速化 (PyTorch 2.0+)
if hasattr(torch, 'compile'):
    model = torch.compile(model)
    print("✅ torch.compile() 有効化")

criterion = nn.CTCLoss(blank=NUM_CLASSES - 1, zero_infinity=True)
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

scheduler = OneCycleLR(
    optimizer,
    max_lr=LR * 10,
    epochs=EPOCHS,
    steps_per_epoch=len(train_loader),
    pct_start=0.1,
    anneal_strategy='cos'
)

print(f"\n[v2 高速化設定]")
print(f"バッチサイズ: {BATCH_SIZE}")
print(f"num_workers: 4, pin_memory: True")
print(f"Mixed Precision (AMP): 有効")

In [ ]:
from torch.cuda.amp import autocast, GradScaler

def train_epoch(model, dataloader, criterion, optimizer, scheduler, device, scaler):
    """
    1エポック分の学習（AMP対応で高速化）
    
    Mixed Precision Training:
    - forward passをFP16で実行（高速化 + メモリ削減）
    - backward passはFP32で実行（精度維持）
    - GradScalerでgradientのunderflow防止
    """
    model.train()
    total_loss = 0.0
    
    for batch in dataloader:
        images = batch["images"].to(device)
        labels = batch["labels"].to(device)
        label_lengths = batch["label_lengths"]
        
        optimizer.zero_grad()
        
        # Mixed Precision: forward pass をFP16で実行
        with autocast():
            outputs = model(images)  # (T, B, C)
            T, B, C = outputs.shape
            input_lengths = torch.full((B,), T, dtype=torch.long)
            loss = criterion(outputs, labels, input_lengths, label_lengths)
        
        # Backward with gradient scaling
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)


def validate(model, dataloader, device):
    """検証"""
    model.eval()
    total_chars = 0
    total_errors = 0
    correct = 0
    total = 0
    samples = []
    
    with torch.no_grad():
        for batch in dataloader:
            images = batch["images"].to(device)
            texts = batch["texts"]
            
            with autocast():
                outputs = model(images)
            
            for i, text in enumerate(texts):
                probs = outputs[:, i, :]
                pred_indices = probs.argmax(dim=1).cpu().tolist()
                pred_text = decode_output(pred_indices)
                
                errors = sum(1 for a, b in zip(text, pred_text) if a != b)
                errors += abs(len(text) - len(pred_text))
                total_chars += len(text)
                total_errors += errors
                
                if text == pred_text:
                    correct += 1
                total += 1
                
                if len(samples) < 5:
                    samples.append({"gt": text, "pred": pred_text, "match": text == pred_text})
    
    cer = (total_errors / total_chars) * 100 if total_chars > 0 else 0
    accuracy = (correct / total) * 100 if total > 0 else 0
    
    return {"cer": cer, "accuracy": accuracy, "samples": samples}

In [ ]:
# 学習ループ (AMP + Early Stopping + 検証頻度削減)
print("=" * 60)
print("学習開始 (Mixed Precision + Early Stopping)")
print("=" * 60)

# GradScaler for AMP
scaler = GradScaler()

best_cer = float("inf")
best_epoch = 0
EARLY_STOP_PATIENCE = 15  # 15エポック改善なしで終了
VAL_FREQUENCY = 5  # 5エポックごとに検証

history = {"train_loss": [], "val_cer": [], "val_acc": [], "lr": []}

start_time = time.time()

for epoch in range(1, EPOCHS + 1):
    epoch_start = time.time()
    
    current_lr = optimizer.param_groups[0]['lr']
    history["lr"].append(current_lr)
    
    # 学習
    train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler, device, scaler)
    history["train_loss"].append(train_loss)
    
    epoch_time = time.time() - epoch_start
    
    # 検証（5エポックごと or 最初の10エポック or 最終エポック）
    if epoch <= 10 or epoch % VAL_FREQUENCY == 0 or epoch == EPOCHS:
        val_result = validate(model, val_loader, device)
        history["val_cer"].append(val_result["cer"])
        history["val_acc"].append(val_result["accuracy"])
        
        print(f"Epoch {epoch:3d}/{EPOCHS} | "
              f"Loss: {train_loss:.4f} | "
              f"CER: {val_result['cer']:.2f}% | "
              f"Acc: {val_result['accuracy']:.1f}% | "
              f"LR: {current_lr:.2e} | "
              f"Time: {epoch_time:.1f}s")
        
        # ベストモデル保存
        if val_result["cer"] < best_cer:
            best_cer = val_result["cer"]
            best_epoch = epoch
            torch.save({
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "cer": best_cer,
                "accuracy": val_result["accuracy"]
            }, "best_model.pth")
            print(f"  -> Best model saved (CER: {best_cer:.2f}%)")
        
        # サンプル表示（20エポックごと）
        if epoch % 20 == 0:
            print("\n  サンプル予測:")
            for s in val_result["samples"][:3]:
                mark = "OK" if s["match"] else "NG"
                print(f"    GT:   {s['gt']}")
                print(f"    Pred: {s['pred']} [{mark}]")
            print()
        
        # Early Stopping チェック
        if epoch - best_epoch >= EARLY_STOP_PATIENCE:
            print(f"\n⚡ Early Stopping: {EARLY_STOP_PATIENCE}エポック改善なし")
            break
    else:
        # 検証スキップ時は簡易ログ
        if epoch % 10 == 0:
            print(f"Epoch {epoch:3d}/{EPOCHS} | Loss: {train_loss:.4f} | Time: {epoch_time:.1f}s (検証スキップ)")

total_time = time.time() - start_time
print("\n" + "=" * 60)
print("学習完了")
print("=" * 60)
print(f"総学習時間: {total_time:.1f}秒 ({total_time/60:.1f}分)")
print(f"ベスト CER: {best_cer:.2f}% (Epoch {best_epoch})")

## 6. ONNX エクスポート

In [ ]:
# ベストモデルをロード
checkpoint = torch.load("best_model.pth")
model.load_state_dict(checkpoint["model_state_dict"])
model.training = False  # set to inference mode

print(f"Loaded model from epoch {checkpoint['epoch']}")
print(f"CER: {checkpoint['cer']:.2f}%, Accuracy: {checkpoint['accuracy']:.1f}%")

In [ ]:
# 依存ライブラリをインストール（ONNX エクスポート用）
!pip install onnx onnxscript onnxruntime -q

# ONNX エクスポート（dynamo=False で旧エクスポーター使用）
# PyTorch 2.9+ の dynamo エクスポーターに BatchNorm 関連のバグがあるため
model_cpu = model.cpu()
model_cpu.training = False
dummy_input = torch.randn(1, 1, 32, 280)

torch.onnx.export(
    model_cpu,
    dummy_input,
    "mrz_crnn.onnx",
    input_names=["image"],
    output_names=["output"],
    dynamic_axes={
        "image": {0: "batch", 3: "width"},
        "output": {0: "seq_len", 1: "batch"}
    },
    opset_version=17,
    dynamo=False  # 旧エクスポーター使用（BatchNorm バグ回避）
)

import os
onnx_size = os.path.getsize("mrz_crnn.onnx") / 1024 / 1024
print(f"ONNX モデルサイズ: {onnx_size:.2f} MB")

In [ ]:
# ONNX モデルの検証
import onnxruntime as ort

session = ort.InferenceSession("mrz_crnn.onnx")

# テスト推論
test_input = np.random.randn(1, 1, 32, 280).astype(np.float32)
outputs = session.run(None, {"image": test_input})

print(f"ONNX 出力形状: {outputs[0].shape}")
print("ONNX モデル検証: OK")

## 7. Google Drive に保存（オプション）

In [ ]:
## まとめ (v2)

### v2 改善点
- **OCR-B フォント**: MRZ標準フォントで合成データ生成
- **データ拡張強化**: ノイズ、ブラー、回転、Perspective、輝度変動、JPEG圧縮
- **AttentionCRNN**: Self-Attention で文字間依存関係を学習
- **正則化強化**: Dropout 0.2、Weight Decay 0.01

### 設定
- **学習データ**: 合成 MRZ 画像 10,000枚 (OCR-B フォント)
- **検証データ**: 合成 MRZ 画像 1,000枚
- **モデルサイズ**: ~3.5 MB (Attention追加分)
- **目標精度**: CER < 0.5%, Accuracy >= 99%

### 次のステップ
1. `mrz_crnn.onnx` をダウンロード
2. WASM 変換 (`onnxruntime-web`)
3. ブラウザでの推論テスト
4. 精度が足りない場合: エポック増加 or データ量増加

## まとめ

- **学習データ**: 合成 MRZ 画像 10,000枚
- **検証データ**: 合成 MRZ 画像 1,000枚
- **モデルサイズ**: ~3 MB
- **目標精度**: CER < 1%

次のステップ:
1. `mrz_crnn.onnx` をダウンロード
2. WASM 変換 (`onnxruntime-web`)
3. ブラウザでの推論テスト